In [5]:
import pickle
loaded_model = pickle.load(open("XGBoostClassifier.pickle.dat", "rb"))

#importing required packages for this module
import pandas as pd
testurldata = pd.read_csv("testing.csv")
testurldata.head(10)

,URLS
0,https://discuss.python.org/t/typeerror-catchin...
1,http://www.csm-testcenter.org/download/malicio...
2,https://economictimes.indiatimes.com/definitio...
3,https://stackoverflow.com/questions/68690135/h...
4,https://teams.microsoft.com/_#/school/conversa...
5,https://support.microsoft.com/en-us/office/unl...
6,https://www.thewindowsclub.com/fix-webcam-keep...
7,https://bazaar.abuse.ch/sample/68daef159f27fef...
8,https://urlhaus.abuse.ch/url/2165900/
9,https://app.schoology.com/login


In [6]:
testurldata.info

<bound method DataFrame.info of                                                 URLS
0  https://discuss.python.org/t/typeerror-catchin...
1  http://www.csm-testcenter.org/download/malicio...
2  https://economictimes.indiatimes.com/definitio...
3  https://stackoverflow.com/questions/68690135/h...
4  https://teams.microsoft.com/_#/school/conversa...
5  https://support.microsoft.com/en-us/office/unl...
6  https://www.thewindowsclub.com/fix-webcam-keep...
7  https://bazaar.abuse.ch/sample/68daef159f27fef...
8              https://urlhaus.abuse.ch/url/2165900/
9                    https://app.schoology.com/login>

In [7]:
testurldata.shape

(10, 1)

In [10]:
from urllib.parse import urlparse
import pandas as pd
import requests
from features import domainAge, domainEnd, forwarding, getDepth, getDomain, getLength, haveAtSign, havingIP, httpDomain, iframe, mouseOver, prefixSuffix, redirection, rightClick, tinyURL
import whois
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib

def web_traffic(url):
    try:
        # Filling the whitespaces in the URL if any
        url = urllib.parse.quote(url)
        # https://www.alexa.com/siteinfo/google.com
        rank = BeautifulSoup(urllib.request.urlopen("https://www.alexa.com/siteinfo/" + url).read(), "xml").find(
            "REACH")['RANK']
        rank = int(rank)
    except Exception:
        return 1
    if rank < 100000:
        return 0
    else:
        return 1

def featureExtraction(url):
    features = []
    # Address bar based features (8)
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(prefixSuffix(url))

    # Domain based features (4)
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1

    features.append(dns)
    features.append(web_traffic(url))
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))

    # HTML & Javascript based features (4)
    try:
        response = requests.get(url, timeout=3)
    except:
        response = ""
    features.append(iframe(response))
    features.append(mouseOver(response))
    features.append(rightClick(response))
    features.append(forwarding(response))
    return features


# Extracting the feautres & storing them in a list
features = []
for i in range(0, 10):
    url = testurldata['URLS'][i]
    print(f'{i+1} {url}')
    features.append(featureExtraction(url))
    

# converting the list to dataframe
feature_names = ['Have_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection',
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                 'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over', 'Right_Click', 'Web_Forwards']

test_urls_features = pd.DataFrame(features, columns=feature_names)
test_urls_features.head()




1 https://discuss.python.org/t/typeerror-catching-classes-that-do-not-inherit-from-baseexception-is-not-allowed/12800/4
2 http://www.csm-testcenter.org/download/malicious/index.html
3 https://economictimes.indiatimes.com/definition/cash-flow
4 https://stackoverflow.com/questions/68690135/how-to-get-the-html-content-from-react-draft-wysiwyg-from-the-child-component-to
5 https://teams.microsoft.com/_#/school/conversations/General?threadId=19:E5VdkeEgOeMmtLUXptT-R67RDhbzmadFtDRmrCJyC241@thread.tacv2&ctx=channel
6 https://support.microsoft.com/en-us/office/unlicensed-product-and-activation-errors-in-office-0d23d3c0-c19c-4b2f-9845-5344fedc4380
7 https://www.thewindowsclub.com/fix-webcam-keeps-turning-off-and-on-again-on-windows-10
8 https://bazaar.abuse.ch/sample/68daef159f27fef712d5bf322affa5c9430d85b2251336c0aee0ae79538f187b/
9 https://urlhaus.abuse.ch/url/2165900/
10 https://app.schoology.com/login


,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards
0,0,0,1,4,0,0,0,0,1,1,1,1,0,0,1,0
1,0,0,1,3,0,0,0,1,1,1,1,1,0,0,1,0
2,0,0,1,2,0,0,0,0,1,1,1,1,0,0,1,0
3,0,0,1,3,0,0,0,0,1,1,1,1,0,0,1,0
4,0,1,1,1,0,0,1,0,1,1,1,1,0,0,1,0


In [ ]:
y_pred = loaded_model.predict(test_urls_features)
print("The output is ",y_pred)